<a href="https://colab.research.google.com/github/emm998/DataScienceEcosystem/blob/main/Spark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e07947031f4897f991d77855a877fe113847e7848e7d038dd3cdfa4b5a6148ba
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pandas as pd

In [3]:
spark = SparkSession.builder.appName('ml-iris').getOrCreate()

In [4]:
spark

In [13]:
from google.colab import files
uploaded = files.upload()

Saving IRIS.csv to IRIS.csv


In [14]:
df = spark.read.csv('IRIS.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- species: string (nullable = true)



In [16]:
print(pd.DataFrame(df.take(5), columns=df.columns).transpose())

                   0       1       2       3       4
sepal_length     5.1     4.9     4.7     4.6     5.0
sepal_width      3.5     3.0     3.2     3.1     3.6
petal_length     1.4     1.4     1.3     1.5     1.4
petal_width      0.2     0.2     0.2     0.2     0.2
species       Setosa  Setosa  Setosa  Setosa  Setosa


In [18]:
print(df.dtypes)

[('sepal_length', 'double'), ('sepal_width', 'double'), ('petal_length', 'double'), ('petal_width', 'double'), ('species', 'string')]


In [20]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'double']

In [21]:
from pyspark.ml.feature import VectorAssembler

In [24]:
numeric_columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
assembler = VectorAssembler(inputCols = numeric_columns, outputCol = "features")
df = assembler.transform(df)
df.show()

+------------+-----------+------------+-----------+-------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|species|         features|
+------------+-----------+------------+-----------+-------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|[5.0,3.4,1.5,0.2]|
|         4.4|        2.9|         1.4|        0.2| Setosa|[4.4,2.9,1.4,0.2]|
|         4.9|        3.1|         1.5|        0.1| Setosa|[4.9,

In [25]:
from pyspark.ml.feature import StringIndexer

In [26]:
labels = StringIndexer(inputCol='species', outputCol='label')
df = labels.fit(df).transform(df)
df.show()

+------------+-----------+------------+-----------+-------+-----------------+-----+
|sepal_length|sepal_width|petal_length|petal_width|species|         features|label|
+------------+-----------+------------+-----------+-------+-----------------+-----+
|         5.1|        3.5|         1.4|        0.2| Setosa|[5.1,3.5,1.4,0.2]|  0.0|
|         4.9|        3.0|         1.4|        0.2| Setosa|[4.9,3.0,1.4,0.2]|  0.0|
|         4.7|        3.2|         1.3|        0.2| Setosa|[4.7,3.2,1.3,0.2]|  0.0|
|         4.6|        3.1|         1.5|        0.2| Setosa|[4.6,3.1,1.5,0.2]|  0.0|
|         5.0|        3.6|         1.4|        0.2| Setosa|[5.0,3.6,1.4,0.2]|  0.0|
|         5.4|        3.9|         1.7|        0.4| Setosa|[5.4,3.9,1.7,0.4]|  0.0|
|         4.6|        3.4|         1.4|        0.3| Setosa|[4.6,3.4,1.4,0.3]|  0.0|
|         5.0|        3.4|         1.5|        0.2| Setosa|[5.0,3.4,1.5,0.2]|  0.0|
|         4.4|        2.9|         1.4|        0.2| Setosa|[4.4,2.9,1.4,0.2]

In [28]:
print(pd.DataFrame(df.take(100), columns=df.columns))

    sepal_length  sepal_width  petal_length  petal_width     species  \
0            5.1          3.5           1.4          0.2      Setosa   
1            4.9          3.0           1.4          0.2      Setosa   
2            4.7          3.2           1.3          0.2      Setosa   
3            4.6          3.1           1.5          0.2      Setosa   
4            5.0          3.6           1.4          0.2      Setosa   
..           ...          ...           ...          ...         ...   
95           5.7          3.0           4.2          1.2  Versicolor   
96           5.7          2.9           4.2          1.3  Versicolor   
97           6.2          2.9           4.3          1.3  Versicolor   
98           5.1          2.5           3.0          1.1  Versicolor   
99           5.7          2.8           4.1          1.3  Versicolor   

                features  label  
0   [5.1, 3.5, 1.4, 0.2]    0.0  
1   [4.9, 3.0, 1.4, 0.2]    0.0  
2   [4.7, 3.2, 1.3, 0.2]    0.0  

In [30]:
train, test = df.randomSplit([0.7, 0.3])

In [31]:
print('Train Size :' + str(train.count()))
print('Test size:' +str(test.count()))

Train Size :107
Test size:43


In [32]:
from pyspark.ml.classification import RandomForestClassifier

In [34]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')

In [35]:
rfModel = rf.fit(train)

In [37]:
predictions = rfModel.transform(test)
predictions.select('sepal_length', 'sepal_width','petal_length', 'petal_width', 'prediction', 'label', 'rawPrediction', 'probability').show(25)

+------------+-----------+------------+-----------+----------+-----+--------------------+--------------------+
|sepal_length|sepal_width|petal_length|petal_width|prediction|label|       rawPrediction|         probability|
+------------+-----------+------------+-----------+----------+-----+--------------------+--------------------+
|         4.4|        3.2|         1.3|        0.2|       0.0|  0.0|      [20.0,0.0,0.0]|       [1.0,0.0,0.0]|
|         4.8|        3.1|         1.6|        0.2|       0.0|  0.0|      [20.0,0.0,0.0]|       [1.0,0.0,0.0]|
|         5.0|        2.0|         3.5|        1.0|       1.0|  1.0|[1.0,18.969696969...|[0.05,0.948484848...|
|         5.0|        3.0|         1.6|        0.2|       0.0|  0.0|      [20.0,0.0,0.0]|       [1.0,0.0,0.0]|
|         5.0|        3.3|         1.4|        0.2|       0.0|  0.0|      [20.0,0.0,0.0]|       [1.0,0.0,0.0]|
|         5.0|        3.5|         1.3|        0.3|       0.0|  0.0|      [20.0,0.0,0.0]|       [1.0,0.0,0.0]|
|

In [40]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [44]:
evaluator = MulticlassClassificationEvaluator(labelCol = 'label', predictionCol = 'prediction')
accuracy = evaluator.evaluate(predictions)

In [46]:
print('Accuracy: %s' % (accuracy))
print('Error: %s' % (1.0 - accuracy))

Accuracy: 0.9530674523834578
Error: 0.0469325476165422


In [51]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

In [52]:
preds = predictions.select(['prediction', 'label']).withColumn('label', F.col('label').cast(FloatType()))
preds = preds.select(['prediction', 'label'])
metrics = MulticlassMetrics(preds.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[13.  0.  0.]
 [ 0. 16.  0.]
 [ 0.  2. 12.]]
